In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import AgglomerativeClustering
from transformers import AutoTokenizer, AutoModel
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
import torch
import nltk
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import string

In [2]:
# Load your German text dataset
# Replace 'your_dataset.csv' with the path to your dataset file
df = pd.read_excel('labeled_data_whole.xlsx')
df

,Unnamed: 0,file_name,text,Sales,filtered_text
0,0,SOM_118_00055_FA163EF3DB12-1b43-e3615700-3dca9...,brot urlaub ihre mein name wie kann ich weiter...,0,NaN
1,1,SOM_TPF_00101_FA163E22F5B7-1af3-de819700-4197c...,einen schönen guten tag lauditag du stehst ja ...,0,lauditag stehst
2,2,SOM_141_00085_FA163E622DEB-1b45-68d0e700-3c9a0...,guten tag und herzlich willkommen bei o sie sp...,0,eta kumpel
3,3,SOM_LUB_00112_FA163E22F5B7-1af3-dc815700-45c46...,hallo herzlich willkommen bei der service sie ...,0,bert bislang
4,4,SOM_AMV_00100_FA163E153AE3-1bb7-3f731700-3fe36...,hallo schönen guten tag mein name die firma ot...,0,NaN
...,...,...,...,...,...
1993,395,SOM_LUB_00246_FA163EF3DB12-1b43-e3615700-3be4f...,herzlich willkommen bei dem otto service sie s...,0,pending eingeschränkt eingeschränkt vorteilen
1994,396,SOM_VYD_00630_FA163ED88855-1bef-b9321700-412dc...,herzlich willkommen bei o mein name wie kann i...,0,bahnhofstraße bahnhof bahnhofstraße ttt inhous...
1995,397,SOM_LUB_00379_FA163E622DEB-1b45-6850d700-3e961...,herzlich willkommen bei o sie sprechen mit ihr...,0,rücksetzung aufgehängt rücksetzung frühen gene...
1996,398,SOM_TPF_00515_FA163E56E95C-1b1d-4929d700-3aab3...,herzlich willkommen bei blau ich werde am buck...,1,mso


In [3]:
# Load the spaCy German language model
nlp = spacy.load('de_core_news_sm')
# Create an empty list to store the updated text without proper names
updated_texts = []
# Iterate over the text column and remove proper names using spaCy
for text in df['text']:
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.pos_ != 'PROPN':
            tokens.append(token.text)
    updated_texts.append(' '.join(tokens))
# Update the 'text' column with the modified text
df['text'] = updated_texts

# Print the updated dataset
df.head()

,Unnamed: 0,file_name,text,Sales,filtered_text
0,0,SOM_118_00055_FA163EF3DB12-1b43-e3615700-3dca9...,brot urlaub ihre mein name wie kann ich weiter...,0,NaN
1,1,SOM_TPF_00101_FA163E22F5B7-1af3-de819700-4197c...,einen schönen guten tag lauditag du stehst ja ...,0,lauditag stehst
2,2,SOM_141_00085_FA163E622DEB-1b45-68d0e700-3c9a0...,guten tag und herzlich willkommen bei sie spre...,0,eta kumpel
3,3,SOM_LUB_00112_FA163E22F5B7-1af3-dc815700-45c46...,herzlich willkommen bei der service sie sprech...,0,bert bislang
4,4,SOM_AMV_00100_FA163E153AE3-1bb7-3f731700-3fe36...,hallo schönen guten tag mein name die firma wa...,0,NaN


In [4]:
# Download the NLTK stop words list and wordnet for lemmatization
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the Snowball stemmer and WordNet lemmatizer
stemmer = SnowballStemmer("german")  # Use the appropriate language for stemming
lemmatizer = WordNetLemmatizer()

# Define a function to preprocess text
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove punctuation and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalnum()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('german'))  # Use the appropriate language
    tokens = [word for word in tokens if word not in stop_words]
    
    # Remove short words (length < 3 characters)
    tokens = [word for word in tokens if len(word) > 2]
    
    # Lemmatize or stem the words
    # Uncomment one of the following lines based on your choice
    # tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    tokens = [stemmer.stem(word) for word in tokens]  # Stemming
    
    # Join the cleaned tokens back into a single string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Apply the preprocessing function to the DataFrame
df['text'] = df['text'].apply(preprocess_text)

# Print the cleaned DataFrame
print(df['text'])


[nltk_data] Downloading package stopwords to C:\Users\Muhammad
[nltk_data]     Farhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Muhammad
[nltk_data]     Farhan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Muhammad
[nltk_data]     Farhan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0       brot urlaub nam weiterhelf leid verkehrt leit ...
1       schon gut tag lauditag steh nam helf geht rufn...
2       gut tag herzlich willkomm sprech leid helf bev...
3       herzlich willkomm servic sprech domin tun kund...
4       hallo schon gut tag nam firma vorlieg wann wan...
                              ...                        
1993    herzlich willkomm servic sprech tun kart momen...
1994    herzlich willkomm nam weiterhelf kurz person k...
1995    herzlich willkomm sprech tarifberat plan tun m...
1996    herzlich willkomm blau buckel nam darf tun bit...
1997    herzlich willkomm servic sprech darf tun frau ...
Name: text, Length: 1998, dtype: object


In [11]:
# Words to remove
words_to_remove = ['bitt', 'mal', 'schon', 'genau', 'dank', 'gut', 'tag', 'numm', 'person', 'kurz', 
                   'einfach', 'neu', 'schau', 'moment', 'sag', 'muss', 'geht', 'fall', 'ganz', 'war', 'gern', 'frag', 
                   'gleich',
                  'guck', 'imm', 'halt', 'wurd', 'seh', 'natur', 'quasi', 'gerad', 'bekomm', 'rout', 'gesagt', 'nein', 
                   'tun', 'syst',
                  'mehr', 'mocht', 'box', 'geh', 'zugangsdat', 'passwort', 'kennwort', 'steht','seit', 'klein', 'bekomm', 'rufnumm', 
                   'kundenkennzahl', 'leid', 'tun', 'nam', 'per', 'willkomm', 'herzlich', 'gemacht', 'bleib', 'herr',
                   'kolleg',
                   'schick', 'frau', 'geb', 'richtig', 'vielleicht', 'viel','kart', 'brot', 'brauch',
                  'erst', 'gibt', 'kommt', 'moglich',  'pro', 'iphon', 'sekund', 'minut', 'wirklich', 'krieg',
                   'tatsach', 'musst', 'weiss',  'darf', 'find', 'gar', 'probl' , 'nee', 'klar']

# Function to remove specified words from text
def remove_specific_words(text):
    # Tokenize the text
    tokens = text.split()
    
    # Filter out words to remove
    filtered_tokens = [word for word in tokens if word not in words_to_remove]
    
    # Join the remaining tokens back into a single string
    cleaned_text = ' '.join(filtered_tokens)
    
    return cleaned_text

# Apply the function to the 'cleaned_text' column
df['text'] = df['text'].apply(remove_specific_words)

# Print the cleaned DataFrame
print(df['text'])


0       urlaub weiterhelf verkehrt leit direkt aufleg ...
1       lauditag steh helf end prepaid registri hinter...
2       sprech helf bevor erklar nenn selbststand leit...
3       servic sprech domin kundennumm hatt sicherheit...
4       hallo firma vorlieg wann wann bestellt komisch...
                              ...                        
1993    servic sprech anruf entschuld angegeb end test...
1994    weiterhelf kennzahl nenn wann termin ungefahr ...
1995    sprech tarifberat plan momentch aktuell wollt ...
1996    blau buckel telefonnumm wiederhol vielmal adre...
1997    servic sprech sofia verstand einzelverbindungs...
Name: text, Length: 1998, dtype: object


In [12]:
# Step 1: Topic Modeling (LDA)
# Convert text data to a document-term matrix
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(df['text'])

In [13]:
# Apply LDA for topic modeling
num_topics = 7  # Adjust the number of topics as needed
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(dtm)

# Get the top words for each topic
def get_top_words(model, vectorizer, n_words=10):
    words = vectorizer.get_feature_names_out()
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        top_word_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [words[i] for i in top_word_idx]
        topics[f"Topic {topic_idx + 1}"] = top_words
    return topics

topics = get_top_words(lda, vectorizer)
print("Top words for each topic:")
for topic, top_words in topics.items():
    print(f"{topic}: {', '.join(top_words)}")

Top words for each topic:
Topic 1: monat, tarif, vertrag, angebot, rabatt, handy, kostet, jahr, bezahl, beispiel
Topic 2: technik, storung, leitung, funktioniert, ticket, anruf, uhr, desweg, adress, versuch
Topic 3: vertrag, rechnung, kundennumm, monat, wunsch, end, wiss, versteh, handy, letzt
Topic 4: vertrag, monat, auftrag, weit, bestat, erhalt, widerrufsbelehr, buch, vertragszusammenfass, sup
Topic 5: post, kundig, vertrag, blau, helf, aktivi, bestat, wart, anruf, soll
Topic 6: versteh, vertrag, adress, storniert, wart, selb, desweg, word, handy, sprech
Topic 7: sms, bewert, wunsch, formular, gesprach, vertrag, hallo, anruf, portier, anbiet
